In [13]:
# import necessary libraries
import itertools
import pandas as pd
import numpy as np
import ast
from sklearn.metrics import silhouette_samples, silhouette_score

In [14]:
def force_format(texts):
    return [str(t) for t in texts]

In [15]:
def compute_word_occurences(texts):
    words = itertools.chain.from_iterable(texts)
    word_count = pd.Series(words).value_counts()
    word_count = pd.DataFrame({"Word": word_count.index, "Count": word_count.values})
    return word_count

In [16]:
def get_l_texts(text_file):
    l_texts=[]
    with open(text_file, "r") as f:
        line = f.readlines()
        list_line = [l.strip() for l in line]
        for l in list_line:
            l_texts.append(ast.literal_eval(l))
    return l_texts

In [17]:
dataset = pd.read_json("News_Category_Dataset_v2.json", lines=True, dtype={"headline": str})

In [18]:
texts = force_format(dataset["headline"])

In [19]:
l_texts = get_l_texts("l_texts.txt")

# BERT

In [20]:
#Import
import bertopic


In [21]:
"""
model= bertopic.BERTopic()
topics = model.fit_transform(l_texts)
model.visualize_topics()
model.get_topic_info()
"""

'\nmodel= bertopic.BERTopic()\ntopics = model.fit_transform(l_texts)\nmodel.visualize_topics()\nmodel.get_topic_info()\n'

In [22]:
model=bertopic.BERTopic()
model=model.load("trained_model_pytorch")

model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,82981,-1_video_mom_photos_school,"[video, mom, photos, school, police, fashion, ...",NaN
1,0,812,0_fitness_workout_gym_exercise,"[fitness, workout, gym, exercise, exercises, w...",NaN
2,1,729,1_food_diet_nutrition_foods,"[food, diet, nutrition, foods, diets, eat, cra...",NaN
3,2,706,2_airlines_airline_flight_passenger,"[airlines, airline, flight, passenger, flights...",NaN
4,3,697,3_climate_change_epa_warming,"[climate, change, epa, warming, deniers, envir...",NaN
...,...,...,...,...,...
2385,2384,10,2384_grammys_grammy_carpet_tacky,"[grammys, grammy, carpet, tacky, bestdressed, ...",NaN
2386,2385,10,2385_despacito_yankee_calculators_overplayed,"[despacito, yankee, calculators, overplayed, s...",NaN
2387,2386,10,2386_stem_subjects_scienceminded_girls,"[stem, subjects, scienceminded, girls, ripples...",NaN
2388,2387,10,2387_alchemists_stutz_phil_piatigorsky,"[alchemists, stutz, phil, piatigorsky, gregor,...",NaN


In [42]:
n_clusters=model.get_topic_info().shape[0]
dict_model=model.get_topics()

for i in range(n_clusters-1):
    print("Cluster ",i)
    list_words=[for dict_model[i]]
    print("\n")

AttributeError: 'list' object has no attribute 'keys'